# Baseline Model Raw

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/saurabhtayal/Capstone-Project---Diabetic-Patients-Readmission-Prediction/master/diabetic_data.csv')

In [ ]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
X = pd.get_dummies(df.drop('readmitted', axis = 1), drop_first = True)
y = df['readmitted'].replace({'NO' : 0, '>30' : 0, '<30' : 1})

In [ ]:
X.head()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Male,gender_Unknown/Invalid,age_[10-20),age_[20-30),age_[30-40),age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),age_[90-100),weight_?,weight_[0-25),weight_[100-125),weight_[125-150),weight_[150-175),weight_[175-200),weight_[25-50),weight_[50-75),weight_[75-100),payer_code_BC,payer_code_CH,...,chlorpropamide_Steady,chlorpropamide_Up,glimepiride_No,glimepiride_Steady,glimepiride_Up,acetohexamide_Steady,glipizide_No,glipizide_Steady,glipizide_Up,glyburide_No,glyburide_Steady,glyburide_Up,tolbutamide_Steady,pioglitazone_No,pioglitazone_Steady,pioglitazone_Up,rosiglitazone_No,rosiglitazone_Steady,rosiglitazone_Up,acarbose_No,acarbose_Steady,acarbose_Up,miglitol_No,miglitol_Steady,miglitol_Up,troglitazone_Steady,tolazamide_Steady,tolazamide_Up,insulin_No,insulin_Steady,insulin_Up,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_Steady,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_Steady,metformin-pioglitazone_Steady,change_No,diabetesMed_Yes
0,2278392,8222157,6,25,1,1,41,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
1,149190,55629189,1,1,7,3,59,0,18,0,0,0,9,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1
2,64410,86047875,1,1,7,2,11,5,13,2,0,1,6,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1
3,500364,82442376,1,1,7,2,44,1,16,0,0,0,7,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1
4,16680,42519267,1,1,7,1,51,0,8,0,0,0,5,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1


In [ ]:
y.value_counts()

0    90409
1    11357
Name: readmitted, dtype: int64

In [ ]:
X.shape

(101766, 2436)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.3)

In [ ]:
lr = LogisticRegression(random_state = 0)

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.0

In [ ]:
confusion_matrix(y_test, y_pred)

array([[27108,     0],
       [ 3422,     0]])

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     27108
           1       0.00      0.00      0.00      3422

    accuracy                           0.89     30530
   macro avg       0.44      0.50      0.47     30530
weighted avg       0.79      0.89      0.84     30530



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 2nd Iteration with some-preprocessed data

## Filling Not Available values in admission columns w.r.t emergencies relative to each other

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/diabetes_half_preprocessed_2.csv')
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits
0,Caucasian,0,[0-10),Not Available,Not Available,Referral,1,41,0,1,0,0,0,Diabetes,Not Required,Not Required,1,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0
1,Caucasian,0,[10-20),Emergency,Discharged to home,Emergency,3,59,0,18,0,0,0,"Endocrine, Nutritional, Metabolic, Immunity",Diabetes,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
2,AfricanAmerican,0,[20-30),Emergency,Discharged to home,Emergency,2,11,5,13,2,0,1,"Pregnancy, Childbirth",Diabetes,External causes of injury,6,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,1,0,3
3,Caucasian,1,[30-40),Emergency,Discharged to home,Emergency,2,44,1,16,0,0,0,Infectious and Parasitic,Diabetes,Circulatory,7,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
4,Caucasian,1,[40-50),Emergency,Discharged to home,Emergency,1,51,0,8,0,0,0,Neoplasms,Neoplasms,Diabetes,5,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,1,1,0,0


In [ ]:
df.shape

(99300, 44)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99300 entries, 0 to 99299
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      97070 non-null  object
 1   gender                    99300 non-null  int64 
 2   age                       99300 non-null  object
 3   admission_type_id         99300 non-null  object
 4   discharge_disposition_id  99300 non-null  object
 5   admission_source_id       99300 non-null  object
 6   time_in_hospital          99300 non-null  int64 
 7   num_lab_procedures        99300 non-null  int64 
 8   num_procedures            99300 non-null  int64 
 9   num_medications           99300 non-null  int64 
 10  number_outpatient         99300 non-null  int64 
 11  number_emergency          99300 non-null  int64 
 12  number_inpatient          99300 non-null  int64 
 13  diag_1                    99300 non-null  object
 14  diag_2                

In [ ]:
df[df['race'].isnull()]['readmitted'].value_counts()

0    2042
1     188
Name: readmitted, dtype: int64

In [ ]:
for i in df.columns:
    print(i, '\n', df[i].value_counts(), '\n')

race 
 Caucasian          74201
AfricanAmerican    18759
Hispanic            2012
Other               1471
Asian                627
Name: race, dtype: int64 

gender 
 0    53439
1    45861
Name: gender, dtype: int64 

age 
 [70-80)     25318
[60-70)     22048
[50-60)     17056
[80-90)     16426
[40-50)      9604
[30-40)      3764
[90-100)     2588
[20-30)      1647
[10-20)       690
[0-10)        159
Name: age, dtype: int64 

admission_type_id 
 Emergency        70504
Elective         18659
Not Available    10137
Name: admission_type_id, dtype: int64 

discharge_disposition_id 
 Discharged to home                             60216
Transferred to another medical facility        20681
Discharged to home with home health service    13002
Not Available                                   4678
Left AMA                                         622
Still patient/referred to this institution       101
Name: discharge_disposition_id, dtype: int64 

admission_source_id 
 Emergency                 

In [ ]:
for i in ['admission_type_id', 'admission_source_id']:
    print(df[i].value_counts())

Emergency        70504
Elective         18659
Not Available    10137
Name: admission_type_id, dtype: int64
Emergency                                        55850
Referral                                         30418
Not Available                                     6853
Transferred from another health care facility     6179
Name: admission_source_id, dtype: int64


In [ ]:
for i in range(len(df['admission_type_id'])):
    if df.loc[i, 'admission_type_id'] == 'Not Available' and df.loc[i, 'admission_source_id'] == 'Emergency':
        df.loc[i, 'admission_type_id'] = 'Emergency'
print(df['admission_type_id'].value_counts())

Emergency        72365
Elective         18659
Not Available     8276
Name: admission_type_id, dtype: int64


In [ ]:
for i in range(len(df['admission_source_id'])):
    if df.loc[i, 'admission_source_id'] == 'Not Available' and df.loc[i, 'admission_type_id'] == 'Emergency':
        df.loc[i, 'admission_source_id'] = 'Emergency'
print(df['admission_source_id'].value_counts())

Emergency                                        56912
Referral                                         30418
Transferred from another health care facility     6179
Not Available                                     5791
Name: admission_source_id, dtype: int64


In [ ]:
for i in range(len(df['admission_type_id'])):
    if df.loc[i, 'admission_type_id'] == 'Not Available' and df.loc[i, 'admission_source_id'] == 'Emergency':
        df.loc[i, 'admission_type_id'] = 'Emergency'
print(df['admission_type_id'].value_counts())

Emergency        72365
Elective         18659
Not Available     8276
Name: admission_type_id, dtype: int64


In [ ]:
for i in range(len(df['admission_source_id'])):
    if df.loc[i, 'admission_source_id'] == 'Not Available' and df.loc[i, 'admission_type_id'] == 'Emergency':
        df.loc[i, 'admission_source_id'] = 'Emergency'
print(df['admission_source_id'].value_counts())

Emergency                                        56912
Referral                                         30418
Transferred from another health care facility     6179
Not Available                                     5791
Name: admission_source_id, dtype: int64


In [ ]:
df[df['admission_type_id'] == 'Emergency']['admission_source_id'].value_counts()

Emergency                                        55991
Referral                                         11513
Transferred from another health care facility     4861
Name: admission_source_id, dtype: int64

In [ ]:
df[df['admission_type_id'] == 'Elective']['admission_source_id'].value_counts()

Referral                                         16432
Transferred from another health care facility     1247
Emergency                                          921
Not Available                                       59
Name: admission_source_id, dtype: int64

In [ ]:
df[df['admission_type_id'] == 'Not Available']['admission_source_id'].value_counts()

Not Available                                    5732
Referral                                         2473
Transferred from another health care facility      71
Name: admission_source_id, dtype: int64

In [ ]:
df[df['admission_source_id'] == 'Emergency']['admission_type_id'].value_counts()

Emergency    55991
Elective       921
Name: admission_type_id, dtype: int64

In [ ]:
df[df['admission_source_id'] == 'Referral']['admission_type_id'].value_counts()

Elective         16432
Emergency        11513
Not Available     2473
Name: admission_type_id, dtype: int64

In [ ]:
df[df['admission_source_id'] == 'Transferred from another health care facility']['admission_type_id'].value_counts()

Emergency        4861
Elective         1247
Not Available      71
Name: admission_type_id, dtype: int64

In [ ]:
df[df['admission_source_id'] == 'Not Available']['admission_type_id'].value_counts()

Not Available    5732
Elective           59
Name: admission_type_id, dtype: int64

In [ ]:
for i in range(len(df['admission_source_id'])):
    if df.loc[i, 'admission_source_id'] == 'Not Available' and df.loc[i, 'admission_type_id'] == 'Elective':
        df.loc[i, 'admission_source_id'] = 'Referral'
print(df['admission_source_id'].value_counts())

Emergency                                        56912
Referral                                         30477
Transferred from another health care facility     6179
Not Available                                     5732
Name: admission_source_id, dtype: int64


## Checking Race Values

### Dropping Nan

In [ ]:
df_2 = df.dropna()
df.shape, df_2.shape

((99300, 44), (97070, 44))

In [ ]:
X = df_2.drop('readmitted', axis = 1)
X = pd.get_dummies(X, drop_first = True)

In [ ]:
y = df_2['readmitted']

In [ ]:
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### Relacing with Mode

In [ ]:
df['race'].isna().sum()

2230

In [ ]:
X = df.drop('readmitted', axis = 1)
X['race'] = X['race'].fillna(X['race'].mode()[0])
print(X['race'].isna().sum())
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']

0


In [ ]:
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013657957244655582

### Filling with probability ratio

In [ ]:
df['race'].isna().sum()

2230

In [ ]:
X = df.drop('readmitted', axis = 1)
vc = X['race'].value_counts(normalize = True)
miss = X['race'].isna()
X.loc[miss, 'race'] = np.random.choice(vc.index, size = miss.sum(), p = vc.values)
X['race'].isna().sum()

0

In [ ]:
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013064133016627079

## Treating Race Column (Dropping Nans) before moving on Age Column

In [ ]:
df['race'].isna().sum(), df.shape

(2230, (99300, 44))

In [ ]:
df = df.dropna()
df['race'].isna().sum(), df.shape

(0, (97070, 44))

In [ ]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'preceding_year_visits'],
      dtype='object')

In [ ]:
for i in ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']:
       print(i, '\n', df[i].value_counts(), '\n')

metformin 
 -2    77697
 0    17777
 1     1035
-1      561
Name: metformin, dtype: int64 

repaglinide 
 -2    95562
 0     1359
 1      106
-1       43
Name: repaglinide, dtype: int64 

nateglinide 
 -2    96389
 0      647
 1       23
-1       11
Name: nateglinide, dtype: int64 

chlorpropamide 
 -2    96987
 0       76
 1        6
-1        1
Name: chlorpropamide, dtype: int64 

glimepiride 
 -2    92105
 0     4466
 1      316
-1      183
Name: glimepiride, dtype: int64 

acetohexamide 
 -2    97069
 0        1
Name: acetohexamide, dtype: int64 

glipizide 
 -2    84827
 0    10963
 1      748
-1      532
Name: glipizide, dtype: int64 

glyburide 
 -2    86820
 0     8925
 1      783
-1      542
Name: glyburide, dtype: int64 

tolbutamide 
 -2    97050
 0       20
Name: tolbutamide, dtype: int64 

pioglitazone 
 -2    89970
 0     6758
 1      226
-1      116
Name: pioglitazone, dtype: int64 

rosiglitazone 
 -2    90907
 0     5913
 1      168
-1       82
Name: rosiglitazone, dty

## Age Values

### Original Categories

In [ ]:
X = df.drop('readmitted', 1)
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numerical Range of Original Categories

In [ ]:
X = df.drop('readmitted', 1)
X['age'].value_counts()

[70-80)     24732
[60-70)     21562
[50-60)     16698
[80-90)     16042
[40-50)      9386
[30-40)      3688
[90-100)     2521
[20-30)      1601
[10-20)       682
[0-10)        158
Name: age, dtype: int64

In [ ]:
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 10,
                             '[20-30)' : 20,
                             '[30-40)' : 30,
                             '[40-50)' : 40,
                             '[50-60)' : 50,
                             '[60-70)' : 60,
                             '[70-80)' : 70,
                             '[80-90)' : 80,
                             '[90-100)' : 90})

In [ ]:
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numeircal Value of Original Categories

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 5,
                             '[10-20)' : 15,
                             '[20-30)' : 25,
                             '[30-40)' : 35,
                             '[40-50)' : 45,
                             '[50-60)' : 55,
                             '[60-70)' : 65,
                             '[70-80)' : 75,
                             '[80-90)' : 85,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numeical Values of Original Categories

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 20,
                             '[20-30)' : 30,
                             '[30-40)' : 40,
                             '[40-50)' : 50,
                             '[50-60)' : 60,
                             '[60-70)' : 70,
                             '[70-80)' : 80,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 5 Categories (0-20) (20-40) (40-60) (60-90) (90-100)

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : '[0-20)',
                             '[10-20)' : '[0-20)',
                             '[20-30)' : '[20-40)',
                             '[30-40)' : '[20-40)',
                             '[40-50)' : '[40-60)',
                             '[50-60)' : '[40-60)',
                             '[60-70)' : '[60-90)',
                             '[70-80)' : '[60-90)',
                             '[80-90)' : '[60-90)',
                             '[90-100)' : '[90-100)'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013550135501355014

Lower Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 90})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numeircal Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 10,
                             '[20-30)' : 30,
                             '[30-40)' : 30,
                             '[40-50)' : 50,
                             '[50-60)' : 50,
                             '[60-70)' : 75,
                             '[70-80)' : 75,
                             '[80-90)' : 75,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 20,
                             '[10-20)' : 20,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 90,
                             '[70-80)' : 90,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories (0-40) (40-90) (90-100)

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : '[0-40)',
                             '[10-20)' : '[0-40)',
                             '[20-30)' : '[0-40)',
                             '[30-40)' : '[0-40)',
                             '[40-50)' : '[40-90)',
                             '[50-60)' : '[40-90)',
                             '[60-70)' : '[40-90)',
                             '[70-80)' : '[40-90)',
                             '[80-90)' : '[40-90)',
                             '[90-100)' : '[90-100)'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 0,
                             '[30-40)' : 0,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 40,
                             '[70-80)' : 40,
                             '[80-90)' : 40,
                             '[90-100)' : 90})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 20,
                             '[10-20)' : 20,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 65,
                             '[50-60)' : 65,
                             '[60-70)' : 65,
                             '[70-80)' : 65,
                             '[80-90)' : 65,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 40,
                             '[10-20)' : 40,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 90,
                             '[50-60)' : 90,
                             '[60-70)' : 90,
                             '[70-80)' : 90,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.014453477868112014

### 3 Categories (0-50) (50-90) (90-100)

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : '[0-50)',
                             '[10-20)' : '[0-50)',
                             '[20-30)' : '[0-50)',
                             '[30-40)' : '[0-50)',
                             '[40-50)' : '[0-50)',
                             '[50-60)' : '[50-90)',
                             '[60-70)' : '[50-90)',
                             '[70-80)' : '[50-90)',
                             '[80-90)' : '[50-90)',
                             '[90-100)' : '[90-100)'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 0,
                             '[30-40)' : 0,
                             '[40-50)' : 0,
                             '[50-60)' : 50,
                             '[60-70)' : 50,
                             '[70-80)' : 50,
                             '[80-90)' : 50,
                             '[90-100)' : 90})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 25,
                             '[10-20)' : 25,
                             '[20-30)' : 25,
                             '[30-40)' : 25,
                             '[40-50)' : 25,
                             '[50-60)' : 70,
                             '[60-70)' : 70,
                             '[70-80)' : 70,
                             '[80-90)' : 70,
                             '[90-100)' : 95})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 50,
                             '[10-20)' : 50,
                             '[20-30)' : 50,
                             '[30-40)' : 50,
                             '[40-50)' : 50,
                             '[50-60)' : 90,
                             '[60-70)' : 90,
                             '[70-80)' : 90,
                             '[80-90)' : 90,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.014453477868112014

### 4 Categories According to Age Range - Child (0-10), Teen (10-20), Adult (20-60), Old (60-100)

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'Child',
                             '[10-20)' : 'Teen',
                             '[20-30)' : 'Adult',
                             '[30-40)' : 'Adult',
                             '[40-50)' : 'Adult',
                             '[50-60)' : 'Adult',
                             '[60-70)' : 'Old',
                             '[70-80)' : 'Old',
                             '[80-90)' : 'Old',
                             '[90-100)' : 'Old'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 10,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 20,
                             '[50-60)' : 20,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 60})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 5,
                             '[10-20)' : 15,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 80,
                             '[70-80)' : 80,
                             '[80-90)' : 80,
                             '[90-100)' : 80})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 20,
                             '[20-30)' : 60,
                             '[30-40)' : 60,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 100,
                             '[70-80)' : 100,
                             '[80-90)' : 100,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories According to Age Range - Young (0-20), Adult (20-60), Old (60-100)

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'Young',
                             '[10-20)' : 'Young',
                             '[20-30)' : 'Adult',
                             '[30-40)' : 'Adult',
                             '[40-50)' : 'Adult',
                             '[50-60)' : 'Adult',
                             '[60-70)' : 'Old',
                             '[70-80)' : 'Old',
                             '[80-90)' : 'Old',
                             '[90-100)' : 'Old'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Lower Numeical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 20,
                             '[30-40)' : 20,
                             '[40-50)' : 20,
                             '[50-60)' : 20,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 60})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Mid Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 10,
                             '[10-20)' : 10,
                             '[20-30)' : 40,
                             '[30-40)' : 40,
                             '[40-50)' : 40,
                             '[50-60)' : 40,
                             '[60-70)' : 80,
                             '[70-80)' : 80,
                             '[80-90)' : 80,
                             '[90-100)' : 80})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 20,
                             '[10-20)' : 20,
                             '[20-30)' : 60,
                             '[30-40)' : 60,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 100,
                             '[70-80)' : 100,
                             '[80-90)' : 100,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories According to Age Range - Young (0-30), Mid (30-60), Old (60-100)

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'Young',
                             '[10-20)' : 'Young',
                             '[20-30)' : 'Young',
                             '[30-40)' : 'Mid',
                             '[40-50)' : 'Mid',
                             '[50-60)' : 'Mid',
                             '[60-70)' : 'Old',
                             '[70-80)' : 'Old',
                             '[80-90)' : 'Old',
                             '[90-100)' : 'Old'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.014453477868112014

Lower Numeical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 0,
                             '[10-20)' : 0,
                             '[20-30)' : 0,
                             '[30-40)' : 30,
                             '[40-50)' : 30,
                             '[50-60)' : 30,
                             '[60-70)' : 60,
                             '[70-80)' : 60,
                             '[80-90)' : 60,
                             '[90-100)' : 60})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013851249623607348

Mid Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 15,
                             '[10-20)' : 15,
                             '[20-30)' : 15,
                             '[30-40)' : 45,
                             '[40-50)' : 45,
                             '[50-60)' : 45,
                             '[60-70)' : 80,
                             '[70-80)' : 80,
                             '[80-90)' : 80,
                             '[90-100)' : 80})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

Higher Numerical Values

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 30,
                             '[10-20)' : 30,
                             '[20-30)' : 30,
                             '[30-40)' : 60,
                             '[40-50)' : 60,
                             '[50-60)' : 60,
                             '[60-70)' : 100,
                             '[70-80)' : 100,
                             '[80-90)' : 100,
                             '[90-100)' : 100})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.013851249623607348

### 5 Categories According to Readmission Rate

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'A',
                             '[10-20)' : 'A',
                             '[20-30)' : 'B',
                             '[30-40)' : 'B',
                             '[40-50)' : 'C',
                             '[50-60)' : 'C',
                             '[60-70)' : 'D',
                             '[70-80)' : 'E',
                             '[80-90)' : 'D',
                             '[90-100)' : 'B'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 4 Categories According to Readmission Rate

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'A',
                             '[10-20)' : 'A',
                             '[20-30)' : 'B',
                             '[30-40)' : 'B',
                             '[40-50)' : 'C',
                             '[50-60)' : 'C',
                             '[60-70)' : 'D',
                             '[70-80)' : 'D',
                             '[80-90)' : 'D',
                             '[90-100)' : 'B'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

### 3 Categories According to Readmission Rate

In [ ]:
X = df.drop('readmitted', 1)
X['age'] = X['age'].replace({'[0-10)' : 'A',
                             '[10-20)' : 'A',
                             '[20-30)' : 'A',
                             '[30-40)' : 'A',
                             '[40-50)' : 'B',
                             '[50-60)' : 'B',
                             '[60-70)' : 'C',
                             '[70-80)' : 'C',
                             '[80-90)' : 'C',
                             '[90-100)' : 'A'})
X = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
lr = LogisticRegression(random_state = 1, solver = 'liblinear')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.3)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
recall_score(y_test, y_pred)

0.01415236374585968

## Treating Age column (Taking Middle Numerical value after making 5 categories)

In [ ]:
df['age'] = df['age'].replace({'[0-10)' : 5,
                             '[10-20)' : 15,
                             '[20-30)' : 25,
                             '[30-40)' : 35,
                             '[40-50)' : 45,
                             '[50-60)' : 55,
                             '[60-70)' : 65,
                             '[70-80)' : 75,
                             '[80-90)' : 85,
                             '[90-100)' : 95})
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits
0,Caucasian,0,5,Not Available,Not Available,Referral,1,41,0,1,0,0,0,Diabetes,Not Required,Not Required,1,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0
1,Caucasian,0,15,Emergency,Discharged to home,Emergency,3,59,0,18,0,0,0,"Endocrine, Nutritional, Metabolic, Immunity",Diabetes,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
2,AfricanAmerican,0,25,Emergency,Discharged to home,Emergency,2,11,5,13,2,0,1,"Pregnancy, Childbirth",Diabetes,External causes of injury,6,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,1,0,3
3,Caucasian,1,35,Emergency,Discharged to home,Emergency,2,44,1,16,0,0,0,Infectious and Parasitic,Diabetes,Circulatory,7,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0
4,Caucasian,1,45,Emergency,Discharged to home,Emergency,1,51,0,8,0,0,0,Neoplasms,Neoplasms,Diabetes,5,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,1,1,0,0


## Creating Features

In [ ]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'preceding_year_visits'],
      dtype='object')

In [ ]:
meds = ['metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']

In [ ]:
len(df)

97070

In [ ]:
df = df.reset_index(drop = True)

### Number of Diabetic Meds

In [ ]:
df['number_diab_meds'] = np.nan
for i in range(len(df)):
    n = 0
    for j in meds:
        if df.loc[i, j] != -2:
            n += 1
    df.loc[i, 'number_diab_meds'] = n

In [ ]:
df['number_diab_meds'].value_counts()

1.0    44940
0.0    22228
2.0    21038
3.0     7504
4.0     1298
5.0       57
6.0        5
Name: number_diab_meds, dtype: int64

### Number of Medicines Changes

In [ ]:
df['number_changes'] = np.nan
for i in range(len(df)):
    n = 0
    for j in meds:
        if df.loc[i, j] == -1 or df.loc[i, j] == 1:
            n += 1
    df.loc[i, 'number_changes'] = n

In [ ]:
df['number_changes'].value_counts()

0.0    70599
1.0    25081
2.0     1279
3.0      106
4.0        5
Name: number_changes, dtype: int64

## Exporting File

In [ ]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,preceding_year_visits,number_diab_meds,number_changes
0,Caucasian,0,5,Not Available,Not Available,Referral,1,41,0,1,0,0,0,Diabetes,Not Required,Not Required,1,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0,0.0,0.0
1,Caucasian,0,15,Emergency,Discharged to home,Emergency,3,59,0,18,0,0,0,"Endocrine, Nutritional, Metabolic, Immunity",Diabetes,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0,1.0,1.0
2,AfricanAmerican,0,25,Emergency,Discharged to home,Emergency,2,11,5,13,2,0,1,"Pregnancy, Childbirth",Diabetes,External causes of injury,6,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,1,0,3,1.0,0.0
3,Caucasian,1,35,Emergency,Discharged to home,Emergency,2,44,1,16,0,0,0,Infectious and Parasitic,Diabetes,Circulatory,7,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,1,1,0,0,1.0,1.0
4,Caucasian,1,45,Emergency,Discharged to home,Emergency,1,51,0,8,0,0,0,Neoplasms,Neoplasms,Diabetes,5,None,None,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,1,1,0,0,2.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97070 entries, 0 to 97069
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   race                      97070 non-null  object 
 1   gender                    97070 non-null  int64  
 2   age                       97070 non-null  int64  
 3   admission_type_id         97070 non-null  object 
 4   discharge_disposition_id  97070 non-null  object 
 5   admission_source_id       97070 non-null  object 
 6   time_in_hospital          97070 non-null  int64  
 7   num_lab_procedures        97070 non-null  int64  
 8   num_procedures            97070 non-null  int64  
 9   num_medications           97070 non-null  int64  
 10  number_outpatient         97070 non-null  int64  
 11  number_emergency          97070 non-null  int64  
 12  number_inpatient          97070 non-null  int64  
 13  diag_1                    97070 non-null  object 
 14  diag_2

In [ ]:
for i in df.columns:
    print(i, '\n', df[i].value_counts(), '\n')

race 
 Caucasian          74201
AfricanAmerican    18759
Hispanic            2012
Other               1471
Asian                627
Name: race, dtype: int64 

gender 
 0    52321
1    44749
Name: gender, dtype: int64 

age 
 75    24732
65    21562
55    16698
85    16042
45     9386
35     3688
95     2521
25     1601
15      682
5       158
Name: age, dtype: int64 

admission_type_id 
 Emergency        70595
Elective         18305
Not Available     8170
Name: admission_type_id, dtype: int64 

discharge_disposition_id 
 Discharged to home                             58751
Transferred to another medical facility        20167
Discharged to home with home health service    12797
Not Available                                   4644
Left AMA                                         610
Still patient/referred to this institution       101
Name: discharge_disposition_id, dtype: int64 

admission_source_id 
 Emergency                                        55994
Referral                       

In [ ]:
df['number_diab_meds'] = df['number_diab_meds'].astype('int64')
df['number_changes'] = df['number_changes'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97070 entries, 0 to 97069
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      97070 non-null  object
 1   gender                    97070 non-null  int64 
 2   age                       97070 non-null  int64 
 3   admission_type_id         97070 non-null  object
 4   discharge_disposition_id  97070 non-null  object
 5   admission_source_id       97070 non-null  object
 6   time_in_hospital          97070 non-null  int64 
 7   num_lab_procedures        97070 non-null  int64 
 8   num_procedures            97070 non-null  int64 
 9   num_medications           97070 non-null  int64 
 10  number_outpatient         97070 non-null  int64 
 11  number_emergency          97070 non-null  int64 
 12  number_inpatient          97070 non-null  int64 
 13  diag_1                    97070 non-null  object
 14  diag_2                

In [ ]:
df['number_diab_meds'].value_counts()

1    44940
0    22228
2    21038
3     7504
4     1298
5       57
6        5
Name: number_diab_meds, dtype: int64

In [ ]:
df['number_changes'].value_counts()

0    70599
1    25081
2     1279
3      106
4        5
Name: number_changes, dtype: int64

In [ ]:
df.to_csv('Diabetes_Preprocessed.csv', index = False)